# Testing the new model

In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel, PeftModelForCausalLM
import time
import pandas as pd
import random
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu, sentence_bleu

/home/ubuntu/proj/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load the model

In [2]:
new_model = "teodor98/LORA_Config_BG_Recepies"
base_model = "NousResearch/Llama-2-7b-chat-hf"


After the training I created a repository on Huggingface where I stored my adapter model.
You can check the following link: [Huggingface repo](https://huggingface.co/teodor98/LORA_Config_BG_Recepies)

In [3]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_quant_type="nf4",
    bnb_8bit_compute_dtype=compute_dtype,
    bnb_8bit_use_double_quant=False,
)

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    #quantization_config=quant_config,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards: 100%|██████████| 2/2 [01:42<00:00, 51.41s/it]


I face a problem with loading the model and the adapter. When I load the base model Llama 7B via quantization I cannot use PeftModel merge_and_unload. I can't merge LORA layers with an already quantized model.

For now, I am loading the whole Llama model and then I am merging it with the fine-tuned adapter.

In [5]:
bg_model = PeftModel.from_pretrained(model, new_model)
bg_model = bg_model.merge_and_unload()

adapter_config.json: 100%|██████████| 453/453 [00:00<00:00, 2.57MB/s]
adapter_model.bin: 100%|██████████| 134M/134M [00:07<00:00, 16.8MB/s] 


In [6]:
!nvidia-smi

Fri Dec 15 14:18:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10G         On   | 00000000:00:1E.0 Off |                    0 |
|  0%   33C    P0    60W / 300W |  14383MiB / 23028MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Because of the problem I my GPU memory grows up to 14GB. Which is a problem if I dont have the optimal infrastructure.

I tried also putting the english model and my adapter in one folder but still it loads only the english base model.

In [7]:
tokenizer = AutoTokenizer.from_pretrained(new_model)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json: 100%|██████████| 695/695 [00:00<00:00, 3.56MB/s]
tokenizer.json: 100%|██████████| 1.84M/1.84M [00:00<00:00, 5.07MB/s]
special_tokens_map.json: 100%|██████████| 434/434 [00:00<00:00, 2.74MB/s]


In [8]:
def generate(text, max_len=200):
  # Ignore warnings
  pipe = pipeline(task="text-generation", model=bg_model, tokenizer=tokenizer, max_length=max_len)
  start_time = time.time()
  logging.set_verbosity(logging.CRITICAL)

  # Run text generation pipeline with our next model
  prompt = text
  result = pipe(f"[INST] {prompt} [/INST]")
  end_time = time.time()
  execution_time = end_time - start_time
  print("Execution time:",execution_time)
  #print(result[0]['generated_text'])

  return result[0]['generated_text']

In [10]:
_ = generate("Какво да си сготвя днес? Имам картофи, месо. подправки.")
_ 

Execution time: 8.639506816864014


'[INST] Какво да си сготвя днес? Имам картофи, месо. подправки. [/INST] Начин на приготвяне на Картофи с месо: Картофите се сваряват и се нарязват на филийки. Месото се сварява и се нарязва на малки парчета. В тава се слагат картофите, месото и подправките. Пече се до готовност. 10 минути преди да се извади от фурната се поръсва с настърган кашкавал. 5 минути преди да се извади от фурната се поръсва с магданоз. 2-3 минути преди да се извади от фурната се поръсва'

In [11]:
_ = generate("Знаеш ли как мога да тренирам невронна мрежа?")
_ 

Execution time: 8.86668872833252


'[INST] Знаеш ли как мога да тренирам невронна мрежа? [/INST] Който е увреден от невронните мрежи, може да се смеси с тях. *похлуп* 10 минути и *похлуп* 10 минути. 10 минути *похлуп* и *похлуп*. 10 минути *похлуп* и *похлуп*. 10 минути *похлуп* и *похлуп*. 10 минути *похлуп* и *похлуп*. 10 минути *похлуп* и *похлуп*. 10 минути *похлуп* и *похлуп*. 10 мину'

This general knowleadge answer is far from correct but we can see that is sounds like recepie (slap in bulgarian means to  cover the pot):

'[INST] Do you know how I can train a neural network? [/INST] Anyone damaged by neural nets can blend in with them. *clap* 10 minutes and *clap* 10 minutes. 10 minutes of *slap* and *slap*. 10 minutes of *slap* and *slap*. 10 minutes of *slap* and *slap*. 10 minutes of *slap* and *slap*. 10 minutes of *slap* and *slap*. 10 minutes of *slap* and *slap*. 10 minutes'

In [22]:
test_data = pd.read_csv("./data/test_data.csv")

test_data

,promped_text
0,<s>[INST] Искам да сготвя нещо хубаво. Имам на...
1,<s>[INST] Гладен съм. Какво мога да си сготвя ...
2,<s>[INST] Гладен съм. Какво мога да си сготвя ...
3,<s>[INST] Гладен съм. Какво мога да си сготвя ...
4,<s>[INST] Искам да сготвя нещо хубаво. Имам на...
...,...
11159,<s>[INST] Искам да сготвя нещо хубаво. Имам на...
11160,<s>[INST] Искам да сготвя нещо хубаво. Имам на...
11161,<s>[INST] Гладен съм. Какво мога да си сготвя ...
11162,<s>[INST] Искам да сготвя нещо хубаво. Имам на...


In [42]:
# Splitting the 'prompted_text' column
test_data[['instruction', 'response']] = test_data['promped_text'].str.split(r'\s*\[/INST\]\s* ', expand=True)

In [43]:
test_data.instruction = test_data.instruction.str.replace("<s>[INST] ", "")
test_data

,promped_text,instruction,response
0,<s>[INST] Искам да сготвя нещо хубаво. Имам на...,Искам да сготвя нещо хубаво. Имам налични след...,Може да приготвиш Яхния от тиквички с джанки: ...
1,<s>[INST] Гладен съм. Какво мога да си сготвя ...,Гладен съм. Какво мога да си сготвя със: кафяв...,Начин на приготвяне на Ливански пилаф с леща и...
2,<s>[INST] Гладен съм. Какво мога да си сготвя ...,Гладен съм. Какво мога да си сготвя със: л пря...,Начин на приготвяне на Домашен катък: Прясното...
3,<s>[INST] Гладен съм. Какво мога да си сготвя ...,Гладен съм. Какво мога да си сготвя със: порто...,Начин на приготвяне на Чийзкейк крем карамел с...
4,<s>[INST] Искам да сготвя нещо хубаво. Имам на...,Искам да сготвя нещо хубаво. Имам налични след...,Може да приготвиш Шницел с кайма: Картофите се...
...,...,...,...
11159,<s>[INST] Искам да сготвя нещо хубаво. Имам на...,Искам да сготвя нещо хубаво. Имам налични след...,Може да приготвиш Кошнички от бекон с плънка: ...
11160,<s>[INST] Искам да сготвя нещо хубаво. Имам на...,Искам да сготвя нещо хубаво. Имам налични след...,Може да приготвиш Руска салата у дома: Всичко ...
11161,<s>[INST] Гладен съм. Какво мога да си сготвя ...,Гладен съм. Какво мога да си сготвя със: карто...,Начин на приготвяне на Млечни картофи: Картофи...
11162,<s>[INST] Искам да сготвя нещо хубаво. Имам на...,Искам да сготвя нещо хубаво. Имам налични след...,Може да приготвиш Лазаня със зеленчуци и риба ...


In [64]:
random.seed(42)

random_numbers = random.sample(range(test_data.shape[0]), 10)

for i in random_numbers:
    picked_txt = test_data.instruction[i]
    print("Instruction:")
    print(picked_txt)
    print("The Real answer is: ")
    print(test_data.response[i] + "\n")
    print("The Generated answer is:")

    gen_txt = generate(picked_txt, max_len=500)
    print(gen_txt.split("[/INST] ")[1])

    print("================================================= \n")

Instruction:
Искам да сготвя нещо хубаво. Имам налични следните продукти: яйца, пудра захар, масло, кисело мляко, брашно, бакпулвер ( пакетче), ванилия, кристална захар, сладко
The Real answer is: 
Може да приготвиш Дамски каприз: Белтъците се отделят от жълтъците. Разбиват се жълтъците с пудрата захар, маслото се разтопява и се добавя към получената смес. Добавят се и киселото мляко, брашното, ванилията и бакпулвера. Разбива се добре, изсипва се в тавичка или формички и се пече до порозовяване. Когато основата е готова се намазва със сладко по избор. Белтъците се разбиват със захарта и този крем се намазва върху сладкото. Запича се леко.

The Generated answer is:
Execution time: 47.139244079589844
Може да приготвиш Мъфини с кристална захар и сладко: Яйцата се разбиват със захарта. Добавят се киселото мляко, разтопеното масло и брашното с бакпулвера. Разбърква се добре и се добавя ванилията. 100 г от сместа се използват за сладкото, а останалата се разпределя в 12 купички. Пекат се в п

You can check out the **English_generations.docx** document which has also the translations of the text. 

As we can see the generations are surprisingly good. Not only I am getting Bulgarian-generated and grammatically correct sentences, but I also get pretty diverse recipes. Sometimes I get a different dish than the one in the test set but also makes sense with all the ingredients. That indicates that the model doesn't overfit as much to learn by hard the answers.
As problem is the repetitiveness of end sentences from the generation. 

In [12]:
generate("Имам шоколад и люти чушки, какво да сготвя?")

Execution time: 8.86569619178772


'[INST] Имам шоколад и люти чушки, какво да сготвя? [/INST] Може да приготвиш Шоколад с люти чушки: Шоколадът се слага в купа и се слага на водна баня докато се разтопи. 1-2 чушки се нарязват на парчета и се слагат в шоколада. По желание може да се добави и канела. Оставя се да се разтопи шоколада и чушките. 2-3 чушки се нарязват на парчета и се слагат в шоколада. Оставя се да се разтопи шоколада и чушките. По желание може да се добави и канела. 4-5 чушки се на'

I asked him to generate a recepie with chocolate and spicy peppers and it generated this:
You can prepare Chocolate with hot peppers: Put the chocolate in a bowl and put it in a water bath until it melts. 1-2 peppers are cut into pieces and put in the chocolate. Cinnamon can be added if desired. Let the chocolate and peppers melt. 2-3 peppers are cut into pieces and put in the chocolate. Let the chocolate and peppers melt. Cinnamon can be added if desired. 4-5 peppers
It generates the same sentence all over again but make the peppers quantity greater and greater. It sounds confident at its recepie


In [13]:
generate("кажи ми как титаник е потънал?")

Execution time: 9.166484117507935


'[INST] кажи ми как титаник е потънал? [/INST] Каквото се е случило с Титаник е достатъчно известно.\nТитаник е пътнически круизен линеен кораб, който е потънал на 15 април 1912 г. в Atlantic Ocean. Потъналият се Титаник е най-големият и най-добре познат от всички потънали кораби в историята. Потъналият се Титаник е пътнически круизен линеен кораб, с дължина 234 м и ширина 27 м. Потъналият се Титаник е с капацитет 2640 пътника и 800'

I asked him to tell me how Titanic sinked and the answer is following:

What happened to the Titanic is well known.
The Titanic was a passenger cruise liner that sank on April 15, 1912 in the Atlantic Ocean. The sunken Titanic is the largest and most well-known of all sunken ships in history. The Titanic Wreck is a passenger cruise liner, 234m long and 27m wide. The Titanic Wreck has a capacity of 2,640 passengers and 800'

I dont get the concrete answer but just explanation that I dont need that much, but at least it generates gramatically correct sentances.

In [14]:
generate("генереирай код на java който минава през цикъл със имена на хора и принтира в конзолата: Аз се казвам името на човека, как се казваш ти?")

Execution time: 7.424367666244507


'[INST] генереирай код на java който минава през цикъл със имена на хора и принтира в конзолата: Аз се казвам името на човека, как се казваш ти? [/INST] Може да се добави всички имена на хора в консолата и да се опита да се кара всичките.\n\n1. Аз се казвам името на човека, как се казваш ти?\n2. Аз се казвам името на човека, как се казваш ти?\n3. Аз се казвам името на човека, как се казваш ти?\n4. Аз се казвам името на човека, как се казваш ти?\n5. Аз се ка'

Here we can see that the model does not generate any kind of Java code:

'[INST] generate java code that loops through people's names and prints to the console: My name is person's name, what's your name? [/INST] Can add all people's names to the console and try to fight them all.\n\n1. My name is the person's name, what is your name?\n2. My name is the person's name, what is your name?\n3. My name is the person's name, what is your name?\n4. My name is the person's name, what is your name?\n5. I'm going

Lets try asking in english

In [15]:
generate("generate java code that loops through people's names and prints to the console: My name is person's name, what's your name?")

Execution time: 8.50626802444458


'[INST] generate java code that loops through people\'s names and prints to the console: My name is person\'s name, what\'s your name? [/INST]  Sure! Here is an example of how you can loop through people\'s names and print them to the console in Java:\n```\nimport java.util.Scanner;\n\npublic class NameGenerator {\n  public static void main(String[] args) {\n    Scanner scanner = new Scanner(System.in);\n    while (scanner.hasNext()) {\n      String name = scanner.next();\n      System.out.println("My name is " + name);\n      System.out.println("What\'s your name?");\n    }\n  }\n}\n```\nThis code uses a `Scanner` object to read input from the user, and a `while` loop to repeat the process until the user enters nothing.'

'[INST] generate java code that loops through people\'s names and prints to the console: My name is person\'s name, what\'s your name? [/INST]  Sure! Here is an example of how you can loop through people\'s names and print them to the console in Java: 
```import java.util.Scanner;
public class NameGenerator {
      public static void main(String[] args) {
           Scanner scanner = new Scanner(System.in);
            while (scanner.hasNext()) {
                     String name = scanner.next();
                     System.out.println("My name is " + name);
                     System.out.println("What\'s your name?"); }
                      }
                }\```
                
This code uses a `Scanner` object to read input from the user, and a `while` loop to repeat the process until the user enters nothing.'

It is not that accurate but still in english it understands and does way more.

In conclusion, I can say that:
- The model can be fine-tuned on a task-specific job and do a pretty good job at it (at least with a big data set like ours)
- My task was to get Bulgarian grammatically correct sentences but it taught another task as well: Generating recipes!
- Still, the model is not good for general knowledge questions.
- It needs huge GPU capacity because does not allow to quantize the model and the fine-tunned adaptor at the same time.